In [7]:
from opentrons import protocol_api
import time
import json

# metadata
metadata = {
    'protocolName': 'Distribute Peptides 96',
    'author': 'Colin Rathbun <rathbunc@dickinson.edu>',
    'description': 'Distribute a 96 well plate plus 2 substrates into two, 96 well plates.',
    'apiLevel': '2.12'
}

# for now, this only works with two peptides
PEPTIDE_WELLS = ['A1','A2']
FRZ_WELL = 'A12'
PEPTIDE_AMOUNT = 20
LYSATE_AMOUNT = 20
FRZ_AMOUNT = 5

# protocol run function. the part after the colon lets your editor know
# where to look for autocomplete suggestions
def run(protocol: protocol_api.ProtocolContext):
    amounts = {
        'peptides':{
            1:0,
            2:0,
        },
        'frz':0
    }

    # labware
    # load custom plates?
    deepwell_def = json.load(open('../labware/labcon_96_wellplate_2200ul/labcon_96_wellplate_2200ul.json'))
    #use this one for now
    celltreat96_def = json.load(open('../labware/celltreat_96_wellplate_350ul/celltreat_96_wellplate_350ul.json'))

    # this should be compatible with 1 plate as well
    deepwell_plates = [
    protocol.load_labware_from_definition(deepwell_def, 1),
    protocol.load_labware_from_definition(deepwell_def, 2),
    ]
    # The number of well96_plates needs to be two times more than the number of deepwell plates
    well96_plates = [
        protocol.load_labware_from_definition(celltreat96_def, 3),
        protocol.load_labware_from_definition(celltreat96_def, 4),
        protocol.load_labware_from_definition(celltreat96_def, 5),
        protocol.load_labware_from_definition(celltreat96_def, 6),
    ]
    
    well12 = protocol.load_labware('nest_12_reservoir_15ml', 7)

    tip_racks = [
        protocol.load_labware('opentrons_96_tiprack_20ul', 11),
        protocol.load_labware('opentrons_96_tiprack_20ul', 10),
        protocol.load_labware('opentrons_96_tiprack_20ul', 9),
        protocol.load_labware('opentrons_96_tiprack_20ul', 8),
    ]

    # pipettes
    right_pipette = protocol.load_instrument('p20_multi_gen2', 'right', tip_racks=tip_racks)

    # Add peptides to wells first.
    for n, dest_plate in enumerate(well96_plates):
        peptide_well_target = PEPTIDE_WELLS[n%len(PEPTIDE_WELLS)]
        right_pipette.transfer(
            PEPTIDE_AMOUNT, 
            well12.wells_by_name()[peptide_well_target], 
            dest_plate.rows(),
            )
        amounts['peptides'][n%len(PEPTIDE_WELLS)+1] += PEPTIDE_AMOUNT*8*len(well96_plates[n].rows())
    
    # Now distribute the two, 96 well plates of mutants into the 96 blackwell plates.
    for n, dest_plate in enumerate(well96_plates):
        deepwell_plate_target = n//len(deepwell_plates) # TODO is this going to alternate properly??
        right_pipette.transfer(
            LYSATE_AMOUNT,
            deepwell_plates[deepwell_plate_target].wells(),
            [well.top(z=-1) for well in dest_plate.wells()],
            #dest_plate.wells(),
            new_tip='once',
            blow_out=True,
        )
    
    # Wait 5 minutes for things to fully mix
    # Maybe this is when we can add tips to the deck??
    incubate_start = time.perf_counter()
    protocol.pause("Please add tips to ALL empty positions and press RESUME.")
    incubate_end = time.perf_counter()

    # This assumes that we added tips!!
    right_pipette.reset_tipracks()

    # Add additional delay if tips were replaced too quickly
    if incubate_end - incubate_start < 300:
        protocol.delay(seconds=300-(incubate_end-incubate_start))

    # Now add FRZ
    for plate in well96_plates:
        right_pipette.transfer(FRZ_AMOUNT, well12.wells_by_name()[FRZ_WELL], plate.wells(),
            new_tip='always',
            mix_after=(3,20)
        )
    amounts['frz']+=96*len(well96_plates)*FRZ_AMOUNT
    
    print("Amounts of reagents used: ",amounts)


Overwriting distribute_2peptide_2_96well.py


In [6]:
from opentrons.simulate import simulate, format_runlog
from collections import Counter

log, _bundle = simulate(open("./distribute_2peptide_2_96well.py"))
#print(Counter(format_runlog(log).split(" "))['Picking'])
print(format_runlog(log))


Amounts of reagents used:  {'peptides': {1: 2560, 2: 2560}, 'frz': 1920}
Transferring 20.0 from A1 of NEST 12 Well Reservoir 15 mL on 7 to A1 of CELLTREAT 96 Well Plate 350 µL on 3
	Picking up tip from A1 of Opentrons 96 Tip Rack 20 µL on 11
	Aspirating 20.0 uL from A1 of NEST 12 Well Reservoir 15 mL on 7 at 7.6 uL/sec
	Dispensing 20.0 uL into A1 of CELLTREAT 96 Well Plate 350 µL on 3 at 7.6 uL/sec
	Aspirating 20.0 uL from A1 of NEST 12 Well Reservoir 15 mL on 7 at 7.6 uL/sec
	Dispensing 20.0 uL into A2 of CELLTREAT 96 Well Plate 350 µL on 3 at 7.6 uL/sec
	Aspirating 20.0 uL from A1 of NEST 12 Well Reservoir 15 mL on 7 at 7.6 uL/sec
	Dispensing 20.0 uL into A3 of CELLTREAT 96 Well Plate 350 µL on 3 at 7.6 uL/sec
	Aspirating 20.0 uL from A1 of NEST 12 Well Reservoir 15 mL on 7 at 7.6 uL/sec
	Dispensing 20.0 uL into A4 of CELLTREAT 96 Well Plate 350 µL on 3 at 7.6 uL/sec
	Aspirating 20.0 uL from A1 of NEST 12 Well Reservoir 15 mL on 7 at 7.6 uL/sec
	Dispensing 20.0 uL into A5 of CELLTREA

In [7]:
log

[{'level': 0,
  'payload': {'instrument': <InstrumentContext: p20_multi_v2.0 in RIGHT>,
   'locations': [A1 of NEST 12 Well Reservoir 15 mL on 7,
    A1 of CELLTREAT 96 Well Plate 350 µL on 3],
   'volume': 20,
   'source': A1 of NEST 12 Well Reservoir 15 mL on 7,
   'dest': [[A1 of CELLTREAT 96 Well Plate 350 µL on 3,
     A2 of CELLTREAT 96 Well Plate 350 µL on 3,
     A3 of CELLTREAT 96 Well Plate 350 µL on 3,
     A4 of CELLTREAT 96 Well Plate 350 µL on 3,
     A5 of CELLTREAT 96 Well Plate 350 µL on 3,
     A6 of CELLTREAT 96 Well Plate 350 µL on 3,
     A7 of CELLTREAT 96 Well Plate 350 µL on 3,
     A8 of CELLTREAT 96 Well Plate 350 µL on 3,
     A9 of CELLTREAT 96 Well Plate 350 µL on 3,
     A10 of CELLTREAT 96 Well Plate 350 µL on 3,
     A11 of CELLTREAT 96 Well Plate 350 µL on 3,
     A12 of CELLTREAT 96 Well Plate 350 µL on 3],
    [B1 of CELLTREAT 96 Well Plate 350 µL on 3,
     B2 of CELLTREAT 96 Well Plate 350 µL on 3,
     B3 of CELLTREAT 96 Well Plate 350 µL on 3,
   